In [1]:
import os
os.chdir("..")
os.chdir("src")
from data_cleaning import DataCleaning
os.chdir("..")
from bertopic import BERTopic
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic.representation import KeyBERTInspired
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from transformers import pipeline
from cuml.preprocessing import normalize
import torch
from datasets import Dataset
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer


2024-04-14 23:05:29.259505: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-14 23:05:29.260407: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 23:05:29.307402: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 23:05:29.557368: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 23:05:30.176192: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Sobre as métricas de avaliação
https://github.com/dice-group/Palmetto/issues/13

In [3]:
data_cleaning_pipeline = DataCleaning()
df = data_cleaning_pipeline.run()

data = Dataset.from_pandas(df)
docs = data["DS_OBJETO"]

2024-04-14 23:05:34 - INFO - Data loaded!
2024-04-14 23:05:34 - INFO - Null values cleaned!
2024-04-14 23:05:34 - INFO - Data types asserted!
2024-04-14 23:05:34 - INFO - Full data cleaned!


In [4]:
stop_words = []

with open("/home/brunodifranco/mestrado/mestrado-ufrgs-cmp617-tce/src/utils/stop_words.txt", "r") as file:
    for row in file:
        stop_words.append(row.strip())

vectorizer_model = CountVectorizer(
    stop_words=stop_words,
    strip_accents="unicode"
)

In [5]:
embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"

sentence_model = SentenceTransformer(model_name_or_path=embedding_model, device=device)
embeddings = sentence_model.encode(docs, show_progress_bar=True)

Batches:   0%|          | 0/4057 [00:00<?, ?it/s]

In [6]:
embeddings = normalize(embeddings)

In [7]:
min_topic_size = 500
umap_model = UMAP(n_components=5, n_neighbors=12, min_dist=0.7)
hdbscan_model = HDBSCAN(min_cluster_size=min_topic_size, min_samples=11, gen_min_span_tree=True, prediction_data=True)

## TESTAR O LDA COM 7 top_n_words (rodar o lda_opt assim, 4 stemmers)
e depois pegar o melhor resultado disso e plotar o word cloud, e tbm calcular as outras metricas

## depois otimizar o BERTTOPIC COM 7 topnwords tbm

In [17]:
# Pass the above models to be used in BERTopic
topic_model = BERTopic(
    nr_topics="auto",
    top_n_words=7,
    embedding_model=embedding_model,
    language="brazilian portuguese",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    representation_model=KeyBERTInspired(),
)

In [18]:
topics, probs = topic_model.fit_transform(docs, embeddings)

In [10]:
# topic_model.get_topics()

In [11]:
# topic_model = BERTopic(
#     nr_topics="auto",
#     embedding_model="paraphrase-multilingual-MiniLM-L12-v2", 
#     language="brazilian portuguese",
#     min_topic_size=500,
#     representation_model=KeyBERTInspired(),
#     vectorizer_model=vectorizer_model

# )
# topics, probs = topic_model.fit_transform(docs)

# IMPORTANTE

- Testar outros modelos de embeddings, de preferencia alguns mais pesados

In [20]:
cleaned_docs = topic_model._preprocess_text(docs)
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]

dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topic_model.get_topics()
topics.pop(-1, None)

topic_words = [
        [word for word, _ in topic_model.get_topic(topic) if word != ""]
        for topic in topics
]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                          texts=tokens, 
                          corpus=corpus,
                          dictionary=dictionary, 
                          coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.8557338177734306

In [21]:
topics_red = topic_model.reduce_topics(docs, nr_topics=7)

In [3]:
device

NameError: name 'device' is not defined

In [2]:
# topics_red.get_topics()

In [23]:
cleaned_docs = topics_red._preprocess_text(docs)
vectorizer = topics_red.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topics_red.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topics_red.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topics_red.get_topic(topic)] 
        for topic in range(len(set(topics))-1)][:10]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                          texts=tokens, 
                          corpus=corpus,
                          dictionary=dictionary, 
                          coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.623689483185865

In [27]:
import optuna
def objective(trial):
    # Parâmetro a ser otimizado: número de tópicos
    nr_topics = trial.suggest_int('nr_topics', 4, 20)

    # Redução de tópicos
    topics_red = topic_model.reduce_topics(docs, nr_topics=nr_topics)
    cleaned_docs = topics_red._preprocess_text(docs)
    vectorizer = topics_red.vectorizer_model
    analyzer = vectorizer.build_analyzer()
    tokens = [analyzer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    topics = topics_red.get_topics()
    topics.pop(-1, None)
    topic_words = [
        [word for word, _ in topics_red.get_topic(topic) if word != ""] for topic in topics
    ]
    topic_words = [[words for words, _ in topics_red.get_topic(topic)] 
                    for topic in range(len(set(topics))-1)][:10]

    # Avaliação da Coerência
    coherence_model = CoherenceModel(topics=topic_words, 
                                     texts=tokens, 
                                     corpus=corpus,
                                     dictionary=dictionary, 
                                     coherence='c_v')
    coherence = coherence_model.get_coherence()

    return coherence

# Otimização com Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)  # Defina o número desejado de tentativas

# Obtendo o melhor número de tópicos e a melhor coerência
best_nr_topics = study.best_params['nr_topics']
best_coherence = study.best_value

print("Melhor número de tópicos:", best_nr_topics)
print("Melhor coerência encontrada:", best_coherence)

[I 2024-04-13 23:07:24,887] A new study created in memory with name: no-name-914aa124-5a61-4743-a62b-6c84116dca52
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if poss

Melhor número de tópicos: 18
Melhor coerência encontrada: 0.4901343860001878
